In [2]:
import pandas as pd
import xlrd
import re
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import datetime
import math

In [13]:
folder_podataka = "./PISpodaci"
pregled = pd.read_excel("../samoPouzdaniPodaciTabela.xlsx", converters= {'kraj': pd.to_datetime, 'pocetak': pd.to_datetime})
preslikavanje_opstina = {"Novi Sad": "NS", "Ruma":"RU","Sombor":"SO"}
preslikavanje_lokacija = {"Cenej": "CE" ,"Novi Slankamen":"NS","Ridjica":"RI","Sremski Karlovci":"SK"}
z=None
#za svaku PIS tabelu, cemo napraviti korespondirajucu PIS tabelu, sa pouzdanim podacima + vrsta i mesto
# VAZNO !! Moraju biti prebaceni u xls format posle skidanja sa PIS sajta. To se moze uraditi otvaranjem u Excelu i cuvsanju u xls formatu
for fname in os.listdir(folder_podataka):
    print (fname)
    #IZVLACENJE AMS naziva iz imena fajla !
    ams =fname.split('_')[0] ## REFACTOR - ovo mora da bude regex. 
    #assert(len(ams)==8)
    print(ams)
    assert(re.match("0+[0-9A-F]+",ams)) # REFACTOR - napravi bolji regex, more robust
    #IZVLACENJE PODATAKA IZ PREGLED TABELE
    record_for_ams = pregled[pregled.ams==ams] #za ovu ams samo trzimo records
    record_for_ams = record_for_ams.fillna(pd.Timestamp.today())#najbolje bi ih bilo napuniti sa poslednjom vrednoscu tabele REFACTOR Ovo znaci da ako nije oznaceno, onda znaci da je jos uvek tamo ovaj, tako da mozemo sve podatke uzeti
    #UCITAVANJE TABELE
    PIStabela=pd.read_excel(os.path.join(folder_podataka,fname),header=[0,1])
    PIStabela = PIStabela.sort_index()
    #flattening the header
    #SREDJIVANJE TABELE!!
    PIStabela.columns = [' '.join(col).strip() for col in PIStabela.columns.values]
    PIStabela = PIStabela.iloc[:,0:3]
    columns_to_drop = []
    assert("Temperature" in col for col in PIStabela.columns)
#z = pd.concat(z.values(),axis=0)
    assert(len(PIStabela.columns)==3)
    #prvo cemo ih iseci, i onda cemo ih spojiti u jednu tabelu !!
    tabele_za_jedan_ams = []
    missing_periods_za_jedan_ams = []
    missing_data_za_ams = []
    #idemo kroz PREGLED tabelu i za svaku stavku odvojim deo PIS tabele, koja je validna BTW
    for index,row in record_for_ams.iterrows(): #dobro je ovo, za ea5 se trazi, to nam treba
        assert(row.pocetak< row.kraj)
        print (row.pocetak,row.kraj)
        izuzet_dan_postavljanja = (row.pocetak+pd.offsets.DateOffset(days=1)) 
        #A dan skidanja????????? mislim da jeste po defaultu
        novi = PIStabela.loc[izuzet_dan_postavljanja:row.kraj].copy() #pravimo novi frejm koji je kopija tog perioda, krecemo posle dana postavljanja, posto za to znamo da su nepouzdani TODO mozda napraviti malo preciznije odsecanje 
        novi["opstina"] = row.opstina #popunjavamo podatke, koje sad znamo za sigurno
        novi["mesto"] = row.mesto
        novi["usev"] = row.usev
        tabele_za_jedan_ams.append(novi) #jos uvek smo kod istog AMS-a, i sve sto nadjemo cemo nakaciti
        #print (novi[pd.isna(novi.iloc[:,0:3][novi.iloc[:,0:3].applymap(np.isreal)])])
        missing_temperatures = novi[novi.isnull().any(axis=1)] # kolone koje imaju nedostajuce vrednosti printamo
        novi.to_csv("pouzdani_%s_%s_%s_%s_%s.csv"%(ams,preslikavanje_opstina[row.opstina],preslikavanje_lokacija[row.mesto],row.usev,row.pocetak))    
        idx_ref = pd.DatetimeIndex(start=novi.index[0], end=novi.index[-1], freq="h")
        gaps = idx_ref[~idx_ref.isin(novi.index)]
        #print (gaps)
        missing = pd.DataFrame(index=gaps)
        missing.to_csv("missing_periods_%s_%s" %(ams+row.opstina+row.mesto +row.usev,row.pocetak))
        missing_periods_za_jedan_ams.append(missing)
        missing_data_za_ams.append(missing_temperatures)
    frejmovi_za_ams = pd.concat(tabele_za_jedan_ams)
    frejmovi_za_ams.to_csv("%s_pouzdaniPeriodi_biljneVrste_lokacije.csv" %(ams))
    missing_periods = pd.concat(missing_periods_za_jedan_ams)
    missing_periods.to_csv("%s_missing.csv" %ams)
    missing_data = pd.concat(missing_data_za_ams)
    missing_data.to_csv("%s_missing_data.csv" %ams)

00000EA5_station_data.xlsx
00000EA5
2010-04-20 00:00:00 2013-03-29 00:00:00
2013-03-29 00:00:00 2019-01-26 19:42:26.782688
00000E94_station_data.xlsx
00000E94
2010-04-16 00:00:00 2019-01-26 19:42:35.939762
00000E9A_station_data.xls
00000E9A
2013-08-02 00:00:00 2018-11-19 00:00:00
